In [ ]:
# Copyright 2024 NASA
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Rice mapping in Bhutan with U-Net using high resolution satellite imagery

### This notebook shows an example of counting the sample size from the `tfrecords`

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/SERVIR/servir-aces/blob/main/notebooks/count_sample_size.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/SERVIR/servir-aces/blob/main/notebooks/count_sample_size.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
</table>
</br>
</br>
</br>

This notebook is also available in this github repo: https://github.com/SERVIR/servir-aces. Navigate to the `notebooks` folder.

## Setup environment

In [1]:
!pip install servir-aces

In [2]:
!git clone https://github.com/SERVIR/servir-aces

Cloning into 'servir-aces'...
remote: Enumerating objects: 731, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 731 (delta 55), reused 40 (delta 40), pack-reused 641
Receiving objects: 100% (731/731), 3.35 MiB | 4.84 MiB/s, done.
Resolving deltas: 100% (478/478), done.


Now the repo is downloaded. We will create an environment file file to place point to our training data and customize parameters for the model. To do this, we make a copy of the `.env.example` file provided.

Under the hood, all the configuration provided via the environment file are parsed as a config object and can be accessed programatically.

Note current version does not expose all the model intracacies through the environment file but future version may include those depending on the need.

In [3]:
!cp servir-aces/.env.example servir-aces/config.env

## Setup config file variables

Okay, now we have the `config.env` file, we will use this to provide our environments and parameters.

Note there are several parameters that can be changed. Let's start by changing the BASEDIR as below.

```
BASEDIR = "/content/"
```

We will download data for this chapter. We will use `datasets` dir to download the data. Let's go ahead and create that.

In [4]:
!mkdir /content/datasets

Let's go ahead and download the datasets for which we need to calculate the number of samples. They can be found at the google cloud storage and we will use `gsutil` to get the dataset in our workspace. Each folder/ dataset has `training`, `testing`, and `validation` subdirectory. Let's start by downloading these datasets in our workspace.

In [5]:
!gsutil -m cp -r gs://dl-book/chapter-1/* /content/datasets

Copying gs://dl-book/chapter-1/dnn_planet_wo_indices/training/training.tfrecord.gz...
Copying gs://dl-book/chapter-1/dnn_planet_wo_indices/testing/testing.tfrecord.gz...
Copying gs://dl-book/chapter-1/.DS_Store...
Copying gs://dl-book/chapter-1/dnn_planet_wo_indices/validation/validation.tfrecord.gz...
Copying gs://dl-book/chapter-1/images/image_202100001.tfrecord.gz...
Copying gs://dl-book/chapter-1/images/image_202100002.tfrecord.gz...
Copying gs://dl-book/chapter-1/images/image_202100003.tfrecord.gz...
Copying gs://dl-book/chapter-1/images/image_202100004.tfrecord.gz...
Copying gs://dl-book/chapter-1/images/image_202100000.tfrecord.gz...
Copying gs://dl-book/chapter-1/images/image_202100005.tfrecord.gz...
Copying gs://dl-book/chapter-1/images/image_2021mixer.json...
Copying gs://dl-book/chapter-1/prediction/prediction_dnn_v1.TFRecord...
==> NOTE: You are downloading one or more large file(s), which would
run significantly faster if you enabled sliced object downloads. This
feature i

We will use the `unet_256x256_planet_wo_indices` dataset inside the `dataset` folder for this exercise. Let's go ahead and change our DATADIR in the `config.env` file as below.

```
DATADIR = "datasets/unet_256x256_planet_wo_indices"
```

These datasets have RGBN from Planetscope mosiac. Since we are trying to map the rice fields, we use growing season and pre-growing season information. Thus, we have 8 optical bands, namely `red_before`, `green_before`, `blue_before`, `nir_before`, `red_during`, `green_during`, `blue_during`, and  `nir_during`. In adidition, you can use `USE_ELEVATION` and `USE_S1` config to include the topographic and radar information. Since currently we are not including these, so we won't be settting these config values. Similarly, these datasets are tiled to 256x256 pixels, so let's also change that.

```
# For model training, USE_ELEVATION extends FEATURES with "elevation" & "slope"
# USE_S1 extends FEATURES with "vv_asc_before", "vh_asc_before", "vv_asc_during", "vh_asc_during",
# "vv_desc_before", "vh_desc_before", "vv_desc_during", "vh_desc_during"
# In case these are not useful and you have other bands in your training data, you can do set
# USE_ELEVATION and USE_S1 to False and update FEATURES to include needed bands
USE_ELEVATION = False
USE_S1 = False

PATCH_SHAPE = (256, 256)
```

Next, we need to calculate the size of the traiing, testing and validation dataset. For this, we know our size before hand. But let's use `aces` useful functionality to calculate this.

```
# Sizes of the training and evaluation datasets.
TRAIN_SIZE = 8531
TEST_SIZE = 1222
VAL_SIZE = 2404
```

## Update the config file programtically

Let's make a dictionary so we can change these config settings programatically.

In [6]:
BASEDIR = "/content/" # @param {type:"string"}
DATADIR = "datasets/unet_256x256_planet_wo_indices" # @param {type:"string"}

USE_ELEVATION = "False" # @param {type:"string"}
USE_S1 = "False" # @param {type:"string"}
PATCH_SHAPE = "(256, 256)" # @param {type:"string"}

BATCH_SIZE = "32" # @param {type:"string"}


In [7]:
config_settings = {
    "BASEDIR" : BASEDIR,
    "DATADIR": DATADIR,
    "USE_ELEVATION": USE_ELEVATION,
    "USE_S1": USE_S1,
    "PATCH_SHAPE": PATCH_SHAPE,
    "BATCH_SIZE": BATCH_SIZE,
}


In [8]:
import dotenv

config_file = "servir-aces/config.env"

for config_key in config_settings:
    dotenv.set_key(dotenv_path=config_file,
                   key_to_set=config_key,
                   value_to_set=config_settings[config_key]
                   )


## Load config file variables

In [9]:
from aces import Config, DataProcessor

In [10]:
config_file = "/content/servir-aces/config.env"
config = Config(config_file, override=True)

BASEDIR: /content
DATADIR: /content/datasets/unet_256x256_planet_wo_indices
using features: ['red_before', 'green_before', 'blue_before', 'nir_before', 'red_during', 'green_during', 'blue_during', 'nir_during']
using labels: ['class']


Most of the config in the `config.env` is now available via the config instance. Let's check few of them here.

In [11]:
config.TRAINING_DIR, config.BATCH_SIZE, config.FEATURES

(PosixPath('/content/datasets/unet_256x256_planet_wo_indices/training'),
 32,
 ['red_before',
  'green_before',
  'blue_before',
  'nir_before',
  'red_during',
  'green_during',
  'blue_during',
  'nir_during'])

## Calculate the number of records

Use the `calculate_n_samples` static function of the `DataProcessor` class to get the number of records for each split. You can provide additional parameters (`PRINT_DATASET`) as well.

In [12]:
%%time
additional_config = {
    "PRINT_DATASET": True
}
n_training_records, n_testing_records, n_validation_records = DataProcessor.calculate_n_samples(**{**config.__dict__, **additional_config})


Training
inputs: float32 (256, 256, 8)
tf.Tensor(
[[[0.04035  0.0461   0.0259   ... 0.03885  0.0167   0.167425]
  [0.03675  0.044525 0.02385  ... 0.0367   0.015075 0.15035 ]
  [0.0288   0.039725 0.022375 ... 0.03295  0.01355  0.144725]
  ...
  [0.029625 0.044825 0.02755  ... 0.0371   0.017125 0.172825]
  [0.027825 0.0424   0.0271   ... 0.036675 0.016825 0.169375]
  [0.029975 0.046125 0.0272   ... 0.037925 0.01715  0.174475]]

 [[0.0328   0.0432   0.0236   ... 0.03615  0.014875 0.159375]
  [0.0273   0.039525 0.0219   ... 0.031475 0.013425 0.139925]
  [0.0321   0.04045  0.02235  ... 0.0346   0.014325 0.152   ]
  ...
  [0.029825 0.0453   0.02725  ... 0.03725  0.01675  0.1777  ]
  [0.0305   0.04465  0.0276   ... 0.03825  0.017225 0.17445 ]
  [0.03075  0.047175 0.027475 ... 0.039175 0.017775 0.17845 ]]

 [[0.029575 0.04125  0.02265  ... 0.03425  0.014675 0.1507  ]
  [0.024    0.038475 0.02175  ... 0.029925 0.012125 0.1312  ]
  [0.031825 0.0415   0.0228   ... 0.03515  0.01495  0.1445  ]
  ..

Instructions for updating:
Use `tf.data.Dataset.ignore_errors` instead.


CPU times: user 3min 47s, sys: 11.2 s, total: 3min 58s
Wall time: 3min 45s


In [13]:
print(f"no of training records: {n_training_records}")
print(f"no of testing records: {n_testing_records}")
print(f"no of validation records: {n_validation_records}")

no of training records: 8531
no of testing records: 1222
no of validation records: 2404
